
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# 転置畳み込み

:label: `sec_transposed_conv`

これまで見てきた畳み込み層 (:numref: `sec_conv_layer` ) やプーリング層 (:numref: `sec_pooling` ) などの CNN 層は、通常、入力の空間次元 (高さと幅) を削減 (ダウンサンプリング) するか、変更しないままにします。 。ピクセルレベルで分類するセマンティックセグメンテーションでは、入力と出力の空間次元が同じであると便利です。たとえば、1 つの出力ピクセルのチャネル次元は、同じ空間位置にある入力ピクセルの分類結果を保持できます。

これを達成するには、特に CNN 層によって空間次元が削減された後、中間特徴マップの空間次元を増加 (アップサンプリング) できる別のタイプの CNN 層を使用できます。このセクションでは、*畳み込みによるダウンサンプリング操作を逆転するための、転置**畳み込み (分数ストライド畳み込み*とも呼ばれます:cite: `Dumoulin.Visin.2016` ) を紹介します。


In [1]:
import torch
from torch import nn
from d2l import torch as d2l


## 基本操作

ここではチャネルを無視して、ストライド 1 でパディングなしの基本的な転置畳み込み演算から始めましょう。 $n_h \times n_w$ 入力テンソルと $k_h \times k_w$ カーネルが与えられたとします。各行で $n_w$ 回、各列で $n_h$ 回、ストライド 1 でカーネル ウィンドウをスライドすると、合計 $n_h n_w$ の中間結果が得られます。各中間結果は、ゼロとして初期化される $(n_h + k_h - 1) \times (n_w + k_w - 1)$ テンソルです。各中間テンソルを計算するには、入力テンソルの各要素がカーネルで乗算され、結果として得られる $k_h \times k_w$ テンソルが各中間テンソルの一部を置き換えます。各中間テンソル内の置換された部分の位置は、計算に使用される入力テンソル内の要素の位置に対応することに注意してください。最終的に、すべての中間結果が合計されて出力が生成されます。

例として、:numref: `fig_trans_conv` 、$2\times 2$ 入力テンソルに対して $2\times 2$ カーネルによる転置畳み込みがどのように計算されるかを示しています。 

![](http://d2l.ai/_images/trans_conv.svg) :label: `fig_trans_conv`

入力行列`X`とカーネル行列`K`に対して`trans_conv`実行できます (**この基本的な転置畳み込み演算を実装できます**)。


In [2]:
def trans_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i: i + h, j: j + w] += X[i, j] * K
    return Y


カーネルを介して入力要素を*減らす*通常の畳み込み (:numref: `sec_conv_layer`内) とは対照的に、転置畳み込みはカーネルを介して入力要素を*ブロードキャストする*ため、入力よりも大きな出力が生成されます。 :numref: `fig_trans_conv`から入力テンソル`X`とカーネル テンソル`K`を構築して、基本的な 2 次元転置畳み込み**演算の [上記の実装の出力を検証**] することができます。


In [3]:
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])


あるいは、入力`X`とカーネル`K`両方とも 4 次元テンソルである場合、[**高レベル API を使用して同じ結果を得る**] ことができます。


In [4]:
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)


## [**パディング、ストライド、および複数のチャネル**]

パディングが入力に適用される通常の畳み込みとは異なり、転置畳み込みではパディングが出力に適用されます。たとえば、高さと幅の両側のパディング番号を 1 に指定すると、最初と最後の行と列が転置された畳み込み出力から削除されます。


In [5]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)


転置畳み込みでは、ストライドは入力ではなく中間結果 (つまり出力) に対して指定されます。 :numref: `fig_trans_conv`からの同じ入力テンソルとカーネル テンソルを使用して、ストライドを 1 から 2 に変更すると、中間テンソルの高さと重みの両方が増加するため、 :numref: `fig_trans_conv_stride2`の出力テンソルが増加します。 

![](http://d2l.ai/_images/trans_conv_stride2.svg) :label: `fig_trans_conv_stride2`

次のコード スニペットは、 :numref: `fig_trans_conv_stride2`のストライド 2 の転置畳み込み出力を検証できます。


In [6]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)


複数の入力および出力チャンネルの場合、転置畳み込みは通常の畳み込みと同じように機能します。入力に ​​$c_i$ チャネルがあり、転置畳み込みによって $k_h\times k_w$ カーネル テンソルが各入力チャネルに割り当てられるとします。複数の出力チャネルが指定されている場合、出力チャネルごとに $c_i\times k_h\times k_w$ カーネルが存在します。

全体として、$\mathsf{X}$ を畳み込み層 $f$ に入力して $\mathsf{Y}=f(\mathsf{X})$ を出力し、転置畳み込み層 $g$ を作成すると、出力チャネルの数が $\mathsf{X}$ のチャネル数であることを除き、$f$ と同じハイパーパラメータの場合、$g(Y)$ は $\mathsf{X}$ と同じ形状になります。これは次の例で説明できます。


In [7]:
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape

True


## 【**マトリックストランスポーズとの接続**】

 :label: `subsec-connection-to-mat-transposition`

転置畳み込みは、行列転置にちなんで名付けられました。説明するために、まず行列の乗算を使用して畳み込みを実装する方法を見てみましょう。以下の例では、 $3\times 3$ 入力`X`と $2\times 2$ 畳み込みカーネル`K`を定義し、関数`corr2d`を使用して畳み込み出力`Y`を計算します。


In [8]:
X = torch.arange(9.0).reshape(3, 3)
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
Y = d2l.corr2d(X, K)
Y

tensor([[27., 37.],
        [57., 67.]])


次に、コンボリューション カーネル`K`多くのゼロを含む疎な重み行列`W`として書き換えます。重み行列の形状は ($4$, $9$) で、非ゼロ要素は畳み込みカーネル`K`から取得されます。


In [9]:
def kernel2matrix(K):
    k, W = torch.zeros(5), torch.zeros((4, 9))
    k[:2], k[3:5] = K[0, :], K[1, :]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W

W = kernel2matrix(K)
W

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])


入力`X`を行ごとに連結して、長さ 9 のベクトルを取得します。次に、 `W`とベクトル化された`X`の行列乗算により、長さ 4 のベクトルが得られます。それを再形成すると、上記の元の畳み込み演算から同じ結果`Y`を取得できます。行列の乗算を使用して畳み込みを実装しただけです。


In [10]:
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])


同様に、行列の乗算を使用して転置畳み込みを実装できます。次の例では、上記の通常の畳み込みからの $2 \times 2$ 出力`Y` 、転置畳み込みへの入力として取得します。行列を乗算することでこの演算を実装するには、重み行列`W`を新しい形状 $(9, 4)$ に転置するだけで済みます。


In [11]:
Z = trans_conv(Y, K)
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])


行列を乗算して畳み込みを実装することを検討してください。入力ベクトル $\mathbf{x}$ と重み行列 $\mathbf{W}$ が与えられると、その入力に重み行列を乗算し、ベクトル $\mathbf{y を出力することで、畳み込みの順伝播関数を実装できます。 }=\mathbf{W}\mathbf{x}$。逆伝播は連鎖則と $\nabla_{\mathbf{x}}\mathbf{y}=\mathbf{W}^\top$ に従うため、畳み込みの逆伝播関数は入力に転置された重みを乗算することで実装できます。行列 $\mathbf{W}^\top$。したがって、転置畳み込み層は、畳み込み層の順伝播関数と逆伝播関数を交換するだけで済みます。その順伝播関数と逆伝播関数は、入力ベクトルに $\mathbf{W}^\top$ と $\mathbf{W} を乗算します。 $、それぞれ。

## まとめ
- カーネルを介して入力要素を減らす通常の畳み込みとは対照的に、転置畳み込みはカーネルを介して入力要素をブロードキャストするため、入力よりも大きな出力が生成されます。
-  $\mathsf{X}$ を畳み込み層 $f$ に入力して $\mathsf{Y}=f(\mathsf{X})$ を出力し、$ と同じハイパーパラメータを持つ転置畳み込み層 $g$ を作成するとします。 $\mathsf{X}$ のチャンネル数である出力チャンネル数を除いて f$ の場合、$g(Y)$ は $\mathsf{X}$ と同じ形状になります。
- 行列の乗算を使用して畳み込みを実装できます。転置畳み込み層は、畳み込み層の順伝播関数と逆伝播関数を交換するだけで済みます。

## 演習
1. :numref: `subsec-connection-to-mat-transposition`では、畳み込み入力`X`と転置された畳み込み出力`Z`同じ形状になります。それらは同じ価値を持っていますか?なぜ？
1. 畳み込みを実装するために行列の乗算を使用するのは効率的ですか?なぜ？



[ディスカッション](https://discuss.d2l.ai/t/1450)
